<a href="https://colab.research.google.com/github/QinyanGong/2024RL/blob/main/Huawei_Research_London_Coding_Interview_Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Coding Test

You will be assessed overall on:

    1) How far you get in the alloted time.
    2) Code optimisations.
    3) Code reusability.
    4) Code readability.



# Transformers

## 1. Coding Test
Below you will find some code which is a classical example of the use of a transformer model.
Your task is to complete the code such that the training runs and the validation loss decreases.

Based on [Attention is all you need, Vaswani et al. 2017](https://arxiv.org/abs/1706.03762), write the code
1. for the multi-head self-attention mechanism
2. for the forward pass of the layer
3. for generating causal masks.

Finally, run the training and validation with the provided functions and dataset.

Note: `pip install portalocker`

In [ ]:
! pip install portalocker
! pip install torchtext==0.12.0
! pip install torchdata==0.3.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 kB 4.0 MB/s eta 0:00:00
  Attempting uninstall: torchdata
    Found existing installation: torchdata 0.4.0
    Uninstalling torchdata-0.4.0:
      Successfully uninstalled torchdata-0.4.0


In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [ ]:
import torch
print(torch.__version__)

1.11.0+cu102


In [ ]:
import os
import time
from copy import deepcopy

import math
from typing import Tuple, Optional
import torch
from torch import Tensor, nn
from torch.utils.data import dataset
from torchtext.datasets import PennTreebank
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

In [ ]:
class MultiHeadSelfAttention(nn.Module):
    def __init__(self, dim: int, n_heads: int, dropout: Optional[float] = None):
        """
        NOTE:   There are different ways of implementing the self-attention.
                In this exercise we assume that the linear projection for the key, query and value
                is defined by a single matrix of shape (dim, dim).
                In your implementation, make the necessary operations such that each head is of equal
                dimension `dim / n_heads`.
                Also make sure that your implementation is compatible with a mask.

        ==================================================
        Creates a multi-head self-attention block
        ==================================================

        :param dim: total dimension of the model (embedding dimension for the input).
        :param n_heads: number of heads in the multi-head attention.
        :param dropout: dropout probability, to avoid overfitting.
        """
        super(MultiHeadSelfAttention, self).__init__()
        self.dim = dim
        self.n_heads = n_heads
        # Each attention head will have a dimension of `dim / n_heads`.
        self.head_dim = dim // n_heads
        assert n_heads * self.head_dim == dim, f"embedding dim={dim} not divisible by n_heads={n_heads}."
        self.dropout = nn.Dropout(p=dropout)
        self.linear_query = nn.Linear(dim, dim)
        self.linear_key = nn.Linear(dim, dim)
        self.linear_value = nn.Linear(dim, dim)
        self.linear_cat_attn = nn.Linear(dim, dim)

    def attention(self, query: Tensor, key: Tensor, value: Tensor, mask: Optional[Tensor] = None) -> Tensor:
        """
        ==================================================
        compute scaled dot-product attention
        ==================================================

        :param query: given sentence that we focused on
        :param key: every sentence to check relationship with qeury
        :param value: every sentence same with Key to get the value
        :param mask: mask for the attention
        :return:
        """
        # key is a 4 dimension tensor
        # [batch_size, head, length, dim_tensor]
        batch_size, head, length, dim_tensor = key.shape

        # 1. dot product query with key^T
        key_transpose = key.transpose(-2,-1)
        scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(dim_tensor)

        # 2. apply masking (optional)
        # if mask is not None:
        #   scores = scores.masked_fill(mask == 0, float('-inf'))
        # 3. softmax
        scores = torch.softmax(scores, dim=-1)

        # 4. multiply with value
        scores = scores @ value

        return scores

    def forward(self, q: Tensor, k: Tensor, v: Tensor, mask: Optional[Tensor] = None) -> Tensor:
        """
        NOTE: the forward pass calls self.attention()

        ==================================================
        compute multi-head self-attention
        ==================================================

        :param q: given sentence that we focused on
        :param k: every sentence to check relationship with qeury
        :param v: every sentence same with Key to get the value
        :param mask: mask for the attention
        :return: multihead self-attention output
        """
        # 1. dot product with weight matrices to get q,k,v
        q = self.linear_query(q)
        k = self.linear_key(k)
        v = self.linear_value(v)

        #  2. split tensor by number of heads [batch_size, length, dim]->[batch_size, head, length, d_tensor]
        split = lambda x: x.view(x.shape[0], x.shape[1], self.n_heads, self.head_dim).transpose(1, 2)
        q = split(q)
        k = split(k)
        v = split(v)

        # 3. compute attention
        attn = self.attention(q, k, v, mask)

        # 4. convert back to [batch_size, length, dim]
        output = attn.transpose(1, 2).contiguous().view(q.size(0), -1, self.dim)
        output = self.linear_cat_attn(output)
        return output

In [ ]:
# test for tensor.view
x = torch.randint(0,100,(1,2,12))
print(x)
# [batch_size, length, dim]->[batch_size, length, head,  d_tensor]
print(x.view(x.shape[0], x.shape[1], 3, 4))
# [batch_size, length, head,  d_tensor]->[batch_size, head, length, d_tensor]
print(x.view(x.shape[0], x.shape[1], 3, 4).transpose(1,2))

tensor([[[73, 68,  5, 56, 81, 35, 93, 50, 69, 65, 92, 91],
         [94, 68, 75, 62, 76, 14, 23, 45, 90, 98, 75, 80]]])
tensor([[[[73, 68,  5, 56],
          [81, 35, 93, 50],
          [69, 65, 92, 91]],

         [[94, 68, 75, 62],
          [76, 14, 23, 45],
          [90, 98, 75, 80]]]])
tensor([[[[73, 68,  5, 56],
          [94, 68, 75, 62]],

         [[81, 35, 93, 50],
          [76, 14, 23, 45]],

         [[69, 65, 92, 91],
          [90, 98, 75, 80]]]])


In [ ]:
class EncoderBlock(nn.Module):
    def __init__(self, attn: MultiHeadSelfAttention, d_model: int, dim_feedforward: int, layer_norm_eps=1e-5, dropout=0.1):
        super(EncoderBlock, self).__init__()
        # Multi-head self-attention
        self.attn = attn

        # Feed-forward network
        self.linear1 = nn.Linear(d_model, dim_feedforward)
        self.dropout = nn.Dropout(dropout)
        self.linear2 = nn.Linear(dim_feedforward, d_model)

        # Layer Normalization
        self.norm1 = nn.LayerNorm(d_model, eps=layer_norm_eps)
        self.norm2 = nn.LayerNorm(d_model, eps=layer_norm_eps)

        # Dropout Layer
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)

        # Activation
        self.activation = nn.ReLU()

    def forward(self, x: Tensor, mask: Optional[Tensor] = None) -> Tensor:
        """
        ==================================================
        create forward pass for the encoder block:
        1. self-attention
        2. add & norm
        3. feed forward
        4. add & norm
        ==================================================

        :param x: input tensor
        :param mask: mask for the attention
        :return: output encoded tensor
        """

        # 1. self-attention
        attn_output = self._self_attn(x, mask)

        # 2. add residual & layer normalization
        x = x + attn_output
        x = self.norm1(x)

        # 3. feed forward
        ff_output = self._feed_forward(x)

        # 4. add residual & layer normalization
        x = x + ff_output
        x = self.norm2(x)

        return x

    def _self_attn(self, x: Tensor, mask: Optional[Tensor]) -> Tensor:
        x = self.attn(q=x, k=x, v=x, mask=mask)
        return self.dropout1(x)

    def _feed_forward(self, x: Tensor) -> Tensor:
        x = self.dropout(self.activation(self.linear1(x)))
        return self.dropout2(self.linear2(x))

In [ ]:
class TransformerEncoder(nn.Module):
    def __init__(self, encoder: EncoderBlock, n_blocks: int):
        super(TransformerEncoder, self).__init__()
        self.encoder_blocks = nn.ModuleList([deepcopy(encoder) for _ in range(n_blocks)])

    def forward(self, x: Tensor, mask: Optional[Tensor] = None) -> Tensor:
        for encoder in self.encoder_blocks:
            x = encoder(x, mask)
        return x

In [ ]:
def generate_square_subsequent_mask(size):
    """
    ==================================================
    create mask for causal attention, where each token in sequence is predicted one by one,
    and the model shouldn't have access to future tokens during the prediction process.
    ==================================================

    :param size: size of the mask
    :return: mask
    """
    # print(size)
    # creates an upper triangular matrix of ones with shape (size,size), and transpose

    mask = (torch.triu(torch.ones(size, size)) == 1). transpose(0, 1)
    # mask with -inf
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))

    return mask

In [ ]:
class TransformerModelManualAttn(nn.Module):

    def __init__(self, ntoken: int, d_model: int, nhead: int, d_hid: int,
                 nlayers: int, dropout: float = 0.5):
        super().__init__()
        # Positional Encoding layer
        self.pos_encoder = PositionalEncodingTorch(d_model, dropout)
        # Encoder Block
        encoder_layers = EncoderBlock(attn=MultiHeadSelfAttention(dim=emsize, n_heads=nhead, dropout=dropout),
                                      d_model=d_model, dim_feedforward=d_hid, dropout=dropout)
        # Transformer Encoder with nlayers layers
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        # Embedding layer
        self.embedding = nn.Embedding(ntoken, d_model)
        self.d_model = d_model
        # Final Linear layer to map output to vocabulary size
        self.linear = nn.Linear(d_model, ntoken)

        self.init_weights()

    def init_weights(self) -> None:
        initrange = 0.1
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.linear.bias.data.zero_()
        self.linear.weight.data.uniform_(-initrange, initrange)

    def forward(self, src: Tensor, src_mask: Tensor = None) -> Tensor:
        """
        ==================================================
        Forward pass of the transformer encoder
        ==================================================

        :param src: input tensor
        :param src_mask: mask for the attention
        :return: output tensor
        """
        # token embedding + positional encoding
        src = self.embedding(src)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src, src_mask)
        output = self.linear(output)
        return output

The code below is given as-is, please contact the examinator if there is any issue running this, unrelated to your code above.

In [ ]:
class PositionalEncodingTorch(nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: Tensor) -> Tensor:
        """
        Arguments:
            x: Tensor, shape ``[seq_len, batch_size, embedding_dim]``
        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

In [ ]:
def data_process(raw_text_iter: dataset.IterableDataset, vocab, tokenizer) -> Tensor:
    data = [torch.tensor(vocab(tokenizer(item)), dtype=torch.long) for item in raw_text_iter]
    return torch.cat(tuple(filter(lambda t: t.numel() > 0, data)))


def batchify(data: Tensor, bsz: int, device: torch.device = None) -> Tensor:
    seq_len = data.shape[0] // bsz
    data = data[:seq_len * bsz]
    data = data.view(bsz, seq_len).t().contiguous()
    return data.to(device)


def get_batch(source: Tensor, i: int, bptt: int) -> Tuple[Tensor, Tensor]:
    seq_len = min(bptt, len(source) - 1 - i)
    data = source[i:i + seq_len]
    target = source[i + 1:i + 1 + seq_len].reshape(-1)
    return data, target

In [ ]:
def train(
        model,
        train_data: Tensor,
        bptt: int,
        criterion,
        ntokens: int,
        optimizer: torch.optim.Optimizer,
        scheduler: torch.optim.lr_scheduler,
        epoch: int = 0,
        device: torch.device = None,
        use_causal_mask: bool = True
) -> None:
    model.train()
    total_loss = 0.
    log_interval = 200
    start_time = time.time()
    src_mask = generate_square_subsequent_mask(bptt).to(device) if use_causal_mask else None

    num_batches = len(train_data) // bptt
    for batch, i in enumerate(range(0, train_data.shape[0] - 1, bptt)):
        data, targets = get_batch(train_data, i, bptt=bptt)
        if data.shape[0] < bptt and src_mask is not None:
            src_mask = generate_square_subsequent_mask(data.shape[0]).to(device)
        src_mask = src_mask[:min(data.shape[0],data.shape[1]), :min(data.shape[0],data.shape[1])]

        # print(data.shape)
        # print(src_mask.shape)
        output = model(src=data, src_mask=src_mask)

        loss = criterion(output.view(-1, ntokens), targets)

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        total_loss += loss.item()
        if batch % log_interval == 0 and batch > 0:
            lr = scheduler.get_last_lr()[0]
            ms_per_batch = (time.time() - start_time) * 1000 / log_interval
            cur_loss = total_loss / log_interval
            ppl = math.exp(cur_loss)
            print(f'| epoch {epoch:3d} | {batch:5d}/{num_batches:5d} batches | '
                  f'lr {lr:02.2f} | ms/batch {ms_per_batch:5.2f} | '
                  f'loss {cur_loss:5.2f}'
                  f' | ppl {ppl:8.2f}'
                  )
            total_loss = 0
            start_time = time.time()


def evaluate(
        model,
        eval_data: Tensor,
        bptt: int,
        ntokens: int,
        criterion,
        device: torch.device = None,
        use_causal_mask: bool = True,
) -> float:
    model.eval()
    total_loss = 0.
    src_mask = generate_square_subsequent_mask(bptt).to(device) if use_causal_mask else None
    with torch.no_grad():
        for i in range(0, eval_data.shape[0] - 1, bptt):
            data, targets = get_batch(eval_data, i, bptt=bptt)
            if data.shape[0] < bptt and src_mask is not None:
                src_mask = generate_square_subsequent_mask(data.shape[0]).to(device)
            output = model(data, src_mask=src_mask)
            output_flat = output.view(-1, ntokens)
            total_loss += data.shape[0] * criterion(output_flat, targets).item()
    return total_loss / (len(eval_data) - 1)

In [ ]:
## Load and batch data
train_iter = PennTreebank(split='train')
tokenizer = get_tokenizer('basic_english')
vocab = build_vocab_from_iterator(map(tokenizer, train_iter), specials=['<unk>'])
vocab.set_default_index(vocab['<unk>'])

train_iter, val_iter, test_iter = PennTreebank()
train_data = data_process(train_iter, vocab=vocab, tokenizer=tokenizer)
val_data = data_process(val_iter, vocab=vocab, tokenizer=tokenizer)
test_data = data_process(test_iter, vocab=vocab, tokenizer=tokenizer)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

bptt = 35
batch_size = 20
eval_batch_size = 10
train_data = batchify(train_data, batch_size, device=device)
val_data = batchify(val_data, eval_batch_size, device=device)
test_data = batchify(test_data, eval_batch_size, device=device)

# Model Parameters
ntokens = len(vocab)
emsize = 200
d_hid = 200
nlayers = 2
nhead = 2
dropout = 0.1
use_causal_mask = True

# Create Model
model = TransformerModelManualAttn(ntokens, emsize, nhead, d_hid, nlayers, dropout).to(device)

### Run model
criterion = nn.CrossEntropyLoss()
lr = 5.0  # learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=lr, )
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.95)

best_val_loss = float('inf')
epochs = 10

# Train
home = os.path.join(os.path.expanduser("~"), "transformer_test")
save_dir = os.path.join(os.path.join(home, 'pytorch_example')) # feel free to change path
os.makedirs(save_dir, exist_ok=True)
best_model_params_path = os.path.join(save_dir, "best_model_params.pt")

for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train(model, train_data, bptt, criterion, ntokens, optimizer, scheduler, epoch, device, use_causal_mask)
    val_loss = evaluate(model, val_data, bptt, ntokens, criterion, device, use_causal_mask)
    val_ppl = math.exp(val_loss)
    elapsed = time.time() - epoch_start_time
    print('-' * 89)
    print(f'| end of epoch {epoch:3d} | time: {elapsed:5.2f}s | '
          f'valid loss {val_loss:5.2f} | valid ppl {val_ppl:8.2f}')
    print('-' * 89)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), best_model_params_path)

    scheduler.step()
model.load_state_dict(torch.load(best_model_params_path))  # load best model states

# Test
test_loss = evaluate(model, test_data, bptt, ntokens, criterion, device)
test_ppl = math.exp(test_loss)
print('=' * 89)
print(f'| End of training | test loss {test_loss:5.2f} | '
      f'test ppl {test_ppl:8.2f}')
print('=' * 89)

| epoch   1 |   200/ 1320 batches | lr 5.00 | ms/batch  8.15 | loss  7.76 | ppl  2347.95
| epoch   1 |   400/ 1320 batches | lr 5.00 | ms/batch  7.75 | loss  6.89 | ppl   981.64
| epoch   1 |   600/ 1320 batches | lr 5.00 | ms/batch  8.17 | loss  6.73 | ppl   837.84
| epoch   1 |   800/ 1320 batches | lr 5.00 | ms/batch  8.71 | loss  6.66 | ppl   777.16
| epoch   1 |  1000/ 1320 batches | lr 5.00 | ms/batch  7.78 | loss  6.62 | ppl   750.70
| epoch   1 |  1200/ 1320 batches | lr 5.00 | ms/batch  7.82 | loss  6.61 | ppl   744.71
-----------------------------------------------------------------------------------------
| end of epoch   1 | time: 11.01s | valid loss  6.64 | valid ppl   768.34
-----------------------------------------------------------------------------------------
| epoch   2 |   200/ 1320 batches | lr 4.75 | ms/batch  7.86 | loss  6.63 | ppl   755.74
| epoch   2 |   400/ 1320 batches | lr 4.75 | ms/batch  7.83 | loss  6.62 | ppl   750.81
| epoch   2 |   600/ 1320 batches 

## 2. General knowledge and LLMs

Describe the attention complexity in memory and computation costs. Do you know methods to try to reduce this cost?

### Memory Complexity

Considering a sequence of length N, for each computation, queries, keys and values are calculated with weights of size $(N * dim)$, where dim is the dimension of each vector.

Therefore, the memory complexity is of $O(N^2 * dim)$, i.e. $O(N^2)$, which grows quadratically with sequence length

### Computation Complexity

queries, keys, values, and scores are computed from dot products, so the coputation complexity is of the order $O(N^2 * dim)$, i.e. $O(N^2)$.

### Low cost attentions

Sparse Attention
Top-k Attention

Why use a causal mask in attention? Are there any other interesting masks or patterns that can be used for training or for generation?

Causal mask can force model predict the future without seeing it. Then the model will be trained to predict the next token while avoid 'cheating' by looking at the future tokens.

Other interesting masks include bidirectional mask, like BERT model.